In [0]:
%pip install 'databricks-sdk==0.61.0' 'pyarrow<20' 'databricks-sdk[notebook]' 'databricks-agents==1.2.0' 'mlflow<=3.1' 'mlflow[databricks]' 'databricks-vectorsearch==0.57' 'langchain==0.3.27' 'langchain_core==0.3.74' 'databricks-langchain==0.7.0' 'bs4' 'dotenv' 'psycopg2-binary==2.9.7' 'pgvector==0.2.5'
import os
if os.environ.get("DATABRICKS_RUNTIME_VERSION"):
    dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.widgets.text("embedding_model", "databricks-gte-large-en")
dbutils.widgets.text("database_instance_name", "tannerw-adtech-db")
dbutils.widgets.text("postgres_database_name", "databricks_postgres")
dbutils.widgets.text("llm_model_serving_endpoint_name", "databricks-claude-3-7-sonnet")
dbutils.widgets.text("target_catalog", "tanner_wendland")
dbutils.widgets.text("target_schema", "default")

In [0]:
embedding_model = dbutils.widgets.get("embedding_model")
database_instance_name = dbutils.widgets.get("database_instance_name")
postgres_database_name = dbutils.widgets.get("postgres_database_name")
llm_model_serving_endpoint_name = dbutils.widgets.get("llm_model_serving_endpoint_name")
target_catalog = dbutils.widgets.get("target_catalog")
target_schema = dbutils.widgets.get("target_schema")

In [0]:
from databricks.sdk import WorkspaceClient
import os
from typing import Optional
import uuid

host = os.environ.get("DATABRICKS_HOST")
token = os.environ.get("DATABRICKS_TOKEN")

workspace_client = WorkspaceClient(host=host, token=token)

def get_postgres_connection(
    client: WorkspaceClient,
    db_name: str,
    database_name: Optional[str] = "databricks_postgres"
) -> str:
    """
    Get PostgreSQL connection string using Databricks SDK.

    Args:
        client (WorkspaceClient): The Databricks workspace client.
        db_name (str): The name of the database instance.
        database_name (Optional[str], optional): The name of the database to connect to.
            Defaults to "databricks_postgres".

    Returns:
        str: PostgreSQL connection string.
    """
    database = client.database.get_database_instance(db_name)
    credentials = client.database.generate_database_credential(
        instance_names=[db_name],
        request_id=str(uuid.uuid4())
    )

    # Use POSTGRES_GROUP env var as username if set, otherwise use current user
    postgres_group = os.getenv('POSTGRES_GROUP')
    username = postgres_group if postgres_group else client.current_user.me().user_name

    database_info = {
        "host": database.read_write_dns,
        "port": "5432",
        "database": database_name,
        "username": username,
        "password": credentials.token,
        "ssl_mode": "require"
    }

    database_url = (
        f"postgresql://{database_info['username']}:{database_info['password']}"
        f"@{database_info['host']}:{database_info['port']}/"
        f"{database_info['database']}?sslmode={database_info['ssl_mode']}"
    )

    return database_url

database_url = get_postgres_connection(workspace_client, database_instance_name, postgres_database_name)

In [0]:
import mlflow
mlflow.set_registry_uri("databricks-uc")

## Chain Config

In [0]:
chain_config = {
    "llm_model_serving_endpoint_name": llm_model_serving_endpoint_name,
    "embedding_model": embedding_model,
    "database_instance_name": database_instance_name,
    "postgres_database_name": postgres_database_name,
    "llm_prompt_template": """You are an assistant that answers questions. You have access to a vector search tool that searches previous conversations, but you should primarily use the immediate conversation context provided.

Current conversation context: {context}

Instructions:
1. First, use the current conversation context above to answer questions
2. Only use the vector search tool if the question explicitly refers to information from previous conversations that is NOT in the current context
3. If the current conversation context contains sufficient information to answer the question, do NOT use the vector search tool
4. The vector search tool should be used sparingly, only when the user is clearly asking about something from their chat history that isn't in the current conversation""",
}

## Vector Config

In [0]:
retriever_config = {
    "parameters": {
        "k": 3
    }
}

In [0]:
from typing import Dict

# combine dynamic and static filters for vector search
def create_configurable_with_filters(input: Dict, retriever_config: Dict) -> Dict:
   """
   create configurable object with filters.
   Args:
       input: The input data containing filters.
   Returns:
       A configurable object with filters added to the search_kwargs.
   """
   if "custom_inputs" in input:
       filters = input["custom_inputs"]["filters"]
   else:
       filters = {}
   print(filters)
   configurable = {
       "configurable": {
           "search_kwargs": {
               "k": retriever_config.get("parameters")["k"],
               "filter": filters
           }
       }
   }
   return configurable

## Simple Chains

In [0]:
from typing import Optional, Dict, Any, List
from databricks_langchain import DatabricksEmbeddings
from sqlalchemy import create_engine, text, event
from pgvector.psycopg2 import register_vector

# Use psycopg2 driver explicitly for SQLAlchemy
database_url_sqlalchemy = database_url.replace("postgresql://", "postgresql+psycopg2://")

engine = create_engine(database_url_sqlalchemy, pool_pre_ping=True)

@event.listens_for(engine, "connect")
def _register_vector(dbapi_connection, connection_record):
    # Register pgvector adapter for psycopg2 so Python lists map to the vector type
    register_vector(dbapi_connection)

def pg_vector_similarity_search(
    embeddings: DatabricksEmbeddings,
    query: str,
    k: int = 3,
    filters: Optional[Dict[str, Any]] = None,
) -> str:
    # 1) Embed the query
    query_embedding = embeddings.embed_query(query)

    # 2) Build where clause from filters
    filters = filters or {}
    where_conditions: List[str] = []
    params: Dict[str, Any] = {}

    if "user_name" in filters:
        where_conditions.append("me.user_name = :user_name")
        params["user_name"] = filters["user_name"]

    if "chat_id" in filters:
        where_conditions.append("me.chat_id = :chat_id")
        params["chat_id"] = filters["chat_id"]

    where_clause = ""
    if where_conditions:
        where_clause = "WHERE " + " AND ".join(where_conditions)

    # 3) Query using cosine distance operator (<=>); requires pgvector
    sql = text(f"""
        SELECT
            ch.message_content,
            me.user_name,
            me.chat_id,
            ch.message_type,
            ch.created_at,
            ch.message_order,
            (me.embedding <=> CAST(:query_embedding AS vector)) AS distance
        FROM message_embeddings me
        JOIN chat_history ch ON me.message_id = ch.id
        {where_clause}
        ORDER BY me.embedding <=> CAST(:query_embedding AS vector)
        LIMIT :k
    """)

    # 4) Execute
    with engine.connect() as conn:
        rows = conn.execute(
            sql, {"query_embedding": query_embedding, "k": k, **params}
        ).fetchall()

    # 5) Format context for the prompt
    passages = [f"Passage: {r.message_content}" for r in rows]
    return "\n".join(passages)

In [0]:
from langchain.tools import Tool
from langchain_core.prompts import ChatPromptTemplate
from databricks_langchain.chat_models import ChatDatabricks
from langchain.tools import Tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda

## Load the chain's configuration
model_config = mlflow.models.ModelConfig(development_config=chain_config)

def pg_vector_search_with_filters(embeddings: DatabricksEmbeddings, query: str, input_data: dict = None) -> str:
    input_data = input_data or {}
    filters = input_data.get("custom_inputs", {}).get("filters", {})
    k = retriever_config["parameters"]["k"]
    return pg_vector_similarity_search(embeddings=embeddings, query=query, k=k, filters=filters)

vector_search_tool = Tool(
    name="search_chat_history",
    description="Retrieve chat history from Postgres (pgvector) for the current user; use only if the immediate conversation context is insufficient.",
    func=lambda q: pg_vector_search_with_filters(q, {}),  # default no filters
)

# Updated prompt template for tool-calling agent
prompt = ChatPromptTemplate.from_messages([
    ("system", model_config.get('llm_prompt_template')),
    ("user", "{question}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Our foundation model answering the final prompt
model = ChatDatabricks(
    endpoint=model_config.get("llm_model_serving_endpoint_name"),
    extra_params={"temperature": 0.01, "max_tokens": 500}
)

embeddings = DatabricksEmbeddings(
    endpoint=chain_config["embedding_model"],
    token=os.environ.get("DATABRICKS_TOKEN")
)

agent = create_tool_calling_agent(model, [vector_search_tool], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[vector_search_tool], verbose=True)


def extract_user_query_string(chat_messages_array):
    return chat_messages_array[-1]["content"]

def extract_context_string(chat_messages_array):
    return '\n'.join([f"Role: {message['role']} - Content: {message['content']}" for message in chat_messages_array[:-1]])

# In your rag_chain_with_tool():
def rag_chain_with_tool(input_data: dict) -> str:
    user_query = extract_user_query_string(input_data["messages"])
    def filtered_vector_search(q: str) -> str:
        return pg_vector_search_with_filters(embeddings, q, input_data)

    filtered_tool = Tool(
        name="search_chat_history",
        description="Retreive chat history from this vector database for the current user, use this tool to answer questions that may refer to previous conversations",
        func=filtered_vector_search,
    )

    filtered_agent = create_tool_calling_agent(model, [filtered_tool], prompt)
    filtered_agent_executor = AgentExecutor(agent=filtered_agent, tools=[filtered_tool], verbose=True)
    context = extract_context_string(input_data["messages"])
    result = filtered_agent_executor.invoke({"question": user_query, "context": context})
    return result["output"]

# Create a runnable version of the chain
chain = RunnableLambda(rag_chain_with_tool)

## Test Document Retreival

In [0]:
input_example_no_filter = {"messages": [ {"role": "user", "content": "Look at the chat history"}]}
answer_no_filter = chain.invoke(input_example_no_filter)
print(answer_no_filter)



> Entering new AgentExecutor chain...

Invoking: `search_chat_history` with `all`
responded: I'll help you look at your chat history. To provide you with relevant information, I need to search your previous conversations.

Passage: Based on the information in our current conversation, math rock emerged in the late 1980s and early 1990s. It developed as a subgenre of rock music characterized by complex rhythmic structures, atypical time signatures, and intricate instrumental performances.

The genre evolved from earlier experimental and progressive rock traditions, with bands like Slint being credited as early pioneers of the sound with their influential album "Spiderland."
Passage: # Math Rock

Math rock is a subgenre of rock music that emerged in the late 1980s and early 1990s. It's characterized by complex rhythmic structures, atypical time signatures, angular melodies, and intricate instrumental performances. Here are some key aspects of math rock:

## Musical Characteristics
- **

In [0]:
input_example = {"messages": [ {"role": "user", "content": "What was my chat history idea?"}], "custom_inputs": {"filters": {"user_name": "tanner.wendland@databricks.com"}}}
answer = chain.invoke(input_example)
print(answer)



> Entering new AgentExecutor chain...

Invoking: `search_chat_history` with `chat history idea`
responded: I don't have enough information from our current conversation to know what specific chat history idea you're referring to. To help you better, I'll need to search your previous conversations.

Passage: # Common Data Structures Used in Audience Segmentation

Building on our discussion about audience segmentation technologies, here are the common data structures used in the audience segmentation process:

## User Profiles
- Individual-level data structures containing attributes and behaviors
- Often stored as JSON objects or relational database records
- Include identifiers, demographics, behaviors, and preferences

## Cookies and Device IDs
- Small text files or unique identifiers stored on user devices
- Used to recognize users across sessions and websites
- Examples: first-party cookies, third-party cookies, mobile advertising IDs

## Taxonomies and Hierarchies
- Organized clas

## Chain PY

In [0]:
from mlflow.models.resources import DatabricksVectorSearchIndex, DatabricksServingEndpoint

chain_file_path = os.path.join(os.getcwd(), 'chain_postgres.py')
if not os.path.exists(chain_file_path):
    raise FileNotFoundError(f"Chain file not found at {chain_file_path}")

workspace_url = spark.conf.get("spark.databricks.workspaceUrl")
os.environ['DATABRICKS_HOST'] = f"https://{workspace_url}"
os.environ['DATABRICKS_TOKEN'] = dbutils.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Log the model to MLflow
with mlflow.start_run(run_name="adtech_chat_history_agent_postgres"):
  logged_chain_info = mlflow.langchain.log_model(
          #Note: In classical ML, MLflow works by serializing the model object.  In generative AI, chains often include Python packages that do not serialize.  Here, we use MLflow's new code-based logging, where we saved our chain under the chain notebook and will use this code instead of trying to serialize the object.
          lc_model=os.path.join(os.getcwd(), 'chain_postgres.py'),  # Chain code file e.g., /path/to/the/chain.py 
          model_config=chain_config, # Chain configuration 
          artifact_path="chain_postgres", # Required by MLflow, the chain's code/config are saved in this directory
          input_example=input_example,
          # Specify resources for automatic authentication passthrough
          resources=[
            DatabricksServingEndpoint(endpoint_name=model_config.get("llm_model_serving_endpoint_name"))
          ],
          pip_requirements=[
            "mlflow==3.1.0",
            "databricks-agents==1.2.0",
            "databricks-langchain==0.7.0",
            "langchain==0.3.27",
            "pgvector==0.2.5",
            "psycopg2-binary==2.9.7",
            "pydantic==2.11.7",
            "sqlalchemy==2.0.43",
            "tornado==6.3.2",
          ]
      )

model_name = "chat_history_agent_postgres"
MODEL_NAME_FQN = f"{target_catalog}.{target_schema}.{model_name}"
# Register to UC
uc_registered_model_info = mlflow.register_model(model_uri=logged_chain_info.model_uri, name=MODEL_NAME_FQN)

2025/08/15 15:37:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
🔗 View Logged Model at: https://e2-demo-field-eng.cloud.databricks.com/ml/experiments/3426381942480258/models/m-0a31cd9abbd44818bd0e2a2848e45927?o=1444828305810485
Failed to detach context
Traceback (most recent call last):
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-611482fe-0d7d-429e-a351-f1c51a31e0fe/lib/python3.11/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-611482fe-0d7d-429e-a351-f1c51a31e0fe/lib/python3.11/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
    self._current_context.reset(token)
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7fc78d5eec50> at 0x7fc7809524c0> was created in a different Context
Failed to detach context
Traceback (most recent call last):
  File "/local_disk0/.ephemeral_nfs/e



> Entering new AgentExecutor chain...

Invoking: `search_chat_history` with `chat history idea`
responded: I don't have enough information from the current conversation context to determine what specific chat history idea you're referring to. To help you better, I'll need to search your previous conversations.



Failed to detach context
Traceback (most recent call last):
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-611482fe-0d7d-429e-a351-f1c51a31e0fe/lib/python3.11/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-611482fe-0d7d-429e-a351-f1c51a31e0fe/lib/python3.11/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
    self._current_context.reset(token)
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7fc78d5eec50> at 0x7fc78096d680> was created in a different Context
Failed to detach context
Traceback (most recent call last):
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-611482fe-0d7d-429e-a351-f1c51a31e0fe/lib/python3.11/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-611482fe-0d7d-429e-a351-f1c51a31e0fe/lib/python3.

Passage: # Common Data Structures Used in Audience Segmentation

Building on our discussion about audience segmentation technologies, here are the common data structures used in the audience segmentation process:

## User Profiles
- Individual-level data structures containing attributes and behaviors
- Often stored as JSON objects or relational database records
- Include identifiers, demographics, behaviors, and preferences

## Cookies and Device IDs
- Small text files or unique identifiers stored on user devices
- Used to recognize users across sessions and websites
- Examples: first-party cookies, third-party cookies, mobile advertising IDs

## Taxonomies and Hierarchies
- Organized classification systems for categorizing audience attributes
- Allow for both broad and granular segmentation
- Example: Interest categories organized from general to specific

## Segment Definitions
- Rules-based or algorithmic definitions of audience groups
- Can be simple (e.g., "males aged 25-34") or c

2025/08/15 15:37:38 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-0a31cd9abbd44818bd0e2a2848e45927
2025/08/15 15:37:38 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
Failed to detach context
Traceback (most recent call last):
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-611482fe-0d7d-429e-a351-f1c51a31e0fe/lib/python3.11/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-611482fe-0d7d-429e-a351-f1c51a31e0fe/lib/python3.11/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
    self._current_context.reset(token)
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7fc78d5eec50> at 0x7fc7809eeac0> was created in a different Context
Failed to detach context
Traceback (most recent call last):
  File "/local_disk0/.ephemeral_nfs/envs/p

...

2025/08/15 15:37:45 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


Based on your previous conversations, I can see you've discussed several technical topics related to AdTech, data structures for audience segmentation, and database technologies. However, I don't see a specific "chat history idea" mentioned in the search results.

If you're referring to a specific idea you shared in a previous conversation, could you provide more details about what you're looking for? For example, was it related to audience segmentation technologies, database structures, or perhaps something about Databricks Genie that was mentioned?

> Finished chain.


Registered model 'tanner_wendland.default.chat_history_agent_postgres' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/14 [00:00<?, ?it/s]

🔗 Created version '7' of model 'tanner_wendland.default.chat_history_agent_postgres': https://e2-demo-field-eng.cloud.databricks.com/explore/data/models/tanner_wendland/default/chat_history_agent_postgres/version/7?o=1444828305810485


## Deploy

In [0]:
dbutils.widgets.text("secert_scope", "field-eng", "Secret Scope")
dbutils.widgets.text("secret_key", "app-secret", "Secret Key")

In [0]:
secret_scope = dbutils.widgets.get("secert_scope")
secret_key = dbutils.widgets.get("secret_key")

secret_value = dbutils.secrets.get(scope=secret_scope, key=secret_key)

print(str(secret_value))

[REDACTED]


In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput

workspace_client = WorkspaceClient()

version = uc_registered_model_info.version
serving_endpoint_name = MODEL_NAME_FQN.replace(".", "-")

workspace_url = spark.conf.get("spark.databricks.workspaceUrl")

config = {
        "served_entities": [
            {
                "name": serving_endpoint_name,
                "entity_name": MODEL_NAME_FQN,
                "entity_version": version,
                "workload_size": "Small",
                "scale_to_zero_enabled": True,
                "environment_vars": {
                    'DATABRICKS_HOST': workspace_url,
                    'DATABRICKS_TOKEN': secret_value
                }
            }
        ]
    }

def does_endpoint_exists(endpoint_name):
    try:
        workspace_client.serving_endpoints.get(endpoint_name)
        return True
    except:
        return False

if not does_endpoint_exists(serving_endpoint_name):
    print(f"Creating endpoint {serving_endpoint_name}...")
    workspace_client.serving_endpoints.create_and_wait(
        serving_endpoint_name,
        config=EndpointCoreConfigInput.from_dict(config)
    )
else:
    print(f"Updating endpoint {serving_endpoint_name}...")
    workspace_client.serving_endpoints.update_config_and_wait(
        serving_endpoint_name,
        served_entities=[ServedEntityInput.from_dict(entity) for entity in config['served_entities']]
    )

Updating endpoint tanner_wendland-default-chat_history_agent_postgres...
